# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import io
import glob
import psycopg2
import datetime as dt
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [6]:
filepath = song_files
all_files = []
for rec in filepath:
    if 'checkpoint' not in rec:
        all_files.append(rec)

#print(all_files)

In [7]:
df = pd.DataFrame()
for rec in all_files:
    df1 = pd.DataFrame(pd.read_json(rec,typ="Series")).transpose()
    #print('file - {}, song_id - {}'.format(rec,df1['song_id']))
    df = df.append(df1)


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:

#df['year'].replace({0:None}, inplace=True)
df['year'] = df['year'].where(df['year'] != 0 ,None)
song_data = df[['song_id','title','artist_id','year','duration']].values.tolist()
#Profile data to check if there are duplicates
#df.groupby(['song_id'])['duration'].unique().groupby(['song_id']).count().sum() --check for other fields as well
#df.groupby(['song_id']).sum()
#df[df['song_id'] == 'SOBAYLL12A8C138AF9']
#SOFCHDR12AB01866EF
#df['year'].replace({0:None})


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
for rec in song_data:
    cur.execute(song_table_insert, rec)
    
cur.execute("select count(*) from songs")
row = cur.fetchone()
print(type(row[0]))

#song_data.to_sql('song_data',)

while row:
    print(row)
    row = cur.fetchone()

<class 'int'>
(71,)


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
df- Convert the array to a list and set it to `artist_data`

In [10]:
df['artist_location'] = df['artist_location'].where(df['artist_location'] != '',None)
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()
df['artist_location'].describe()
#for rec in artist_data:
#    if rec[0] == 'ARNNKDK1187B98BBD5':
#        print(str(rec[4]))
#df.groupby('artist_id').sum()
#df[df['artist_id'] == 'ARD7TVE1187B99BFB1']
#df.groupby('artist_id')['artist_longitude'].unique().count()

count                  43
unique                 40
top       California - LA
freq                    2
Name: artist_location, dtype: object

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
for rec in artist_data:
    cur.execute(artist_table_insert, rec)

cur.execute("select count(*) from artists")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(69,)


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data")

In [13]:
filepath= []
for rec in log_files:
    if 'checkpoint' not in rec:
        filepath.append(rec)

filepath.sort()
print(filepath)
    

['/home/workspace/data/log_data/2018/11/2018-11-01-events.json', '/home/workspace/data/log_data/2018/11/2018-11-02-events.json', '/home/workspace/data/log_data/2018/11/2018-11-03-events.json', '/home/workspace/data/log_data/2018/11/2018-11-04-events.json', '/home/workspace/data/log_data/2018/11/2018-11-05-events.json', '/home/workspace/data/log_data/2018/11/2018-11-06-events.json', '/home/workspace/data/log_data/2018/11/2018-11-07-events.json', '/home/workspace/data/log_data/2018/11/2018-11-08-events.json', '/home/workspace/data/log_data/2018/11/2018-11-09-events.json', '/home/workspace/data/log_data/2018/11/2018-11-10-events.json', '/home/workspace/data/log_data/2018/11/2018-11-11-events.json', '/home/workspace/data/log_data/2018/11/2018-11-12-events.json', '/home/workspace/data/log_data/2018/11/2018-11-13-events.json', '/home/workspace/data/log_data/2018/11/2018-11-14-events.json', '/home/workspace/data/log_data/2018/11/2018-11-15-events.json', '/home/workspace/data/log_data/2018/11/

In [14]:
df = pd.DataFrame()
print(len(filepath))
print(filepath)
for rec in filepath:
    df1 = pd.read_json(rec,lines=True)
    df1['filenm'] = str(rec)
    df = df.append(df1)

#df.loc[[0]]

30
['/home/workspace/data/log_data/2018/11/2018-11-01-events.json', '/home/workspace/data/log_data/2018/11/2018-11-02-events.json', '/home/workspace/data/log_data/2018/11/2018-11-03-events.json', '/home/workspace/data/log_data/2018/11/2018-11-04-events.json', '/home/workspace/data/log_data/2018/11/2018-11-05-events.json', '/home/workspace/data/log_data/2018/11/2018-11-06-events.json', '/home/workspace/data/log_data/2018/11/2018-11-07-events.json', '/home/workspace/data/log_data/2018/11/2018-11-08-events.json', '/home/workspace/data/log_data/2018/11/2018-11-09-events.json', '/home/workspace/data/log_data/2018/11/2018-11-10-events.json', '/home/workspace/data/log_data/2018/11/2018-11-11-events.json', '/home/workspace/data/log_data/2018/11/2018-11-12-events.json', '/home/workspace/data/log_data/2018/11/2018-11-13-events.json', '/home/workspace/data/log_data/2018/11/2018-11-14-events.json', '/home/workspace/data/log_data/2018/11/2018-11-15-events.json', '/home/workspace/data/log_data/2018/

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df = df.loc[df['page']=='NextSong']
df['filenm'].count()

6820

In [16]:
t = df['ts'].transform(lambda x:(dt.datetime.fromtimestamp(x/1000)))
t.head(1)

2   2018-11-01 21:01:46.796
Name: ts, dtype: datetime64[ns]

In [17]:
time_data = (t,t.dt.hour,t.dt.day,t.dt.weekofyear,t.dt.month,t.dt.year,t.dt.weekday)
column_labels = ('strt_tm','hour','day','weekofyear','month','year','weekday')

In [18]:
time_df = pd.DataFrame(dict(zip(column_labels,time_data)))
print(time_df.count())
time_df.drop_duplicates(keep='first',inplace=True)
print(time_df.count())

op = io.StringIO()
time_df.to_csv(op,sep = '\t', header = False, index = False)
op.seek(0)
contents = op.getvalue()
#cur.copy_from(op, 'time')
conn.commit()
cur.execute('select * from time')

strt_tm       6820
hour          6820
day           6820
weekofyear    6820
month         6820
year          6820
weekday       6820
dtype: int64
strt_tm       6813
hour          6813
day           6813
weekofyear    6813
month         6813
year          6813
weekday       6813
dtype: int64


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    
cur.execute("select count(*) from time")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(6813,)


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [21]:

df.sort_values(['ts'], inplace = True)
user_df = pd.DataFrame(df[['userId','firstName','lastName','gender','level']])


print(user_df.count())
print(user_df.drop_duplicates(keep='first',inplace=True))
print(user_df[user_df['userId']== '80'])
print(user_df.groupby('userId').count())
print(user_df)
#data profile
#df[df['userId'] == '3']
#df[df['userId'] == '49']
#df[df['ts'] == 1543418559796]['level']
#user_df.groupby(['userId']).count()
#user_df.groupby(['userId'])['level'].unique().groupby(['userId']).count()



userId       6820
firstName    6820
lastName     6820
gender       6820
level        6820
dtype: int64
None
    userId firstName lastName gender level
148     80     Tegan   Levine      F  free
40      80     Tegan   Levine      F  paid
        firstName  lastName  gender  level
userId                                    
3               1         1       1      1
6               1         1       1      1
7               1         1       1      1
8               1         1       1      1
10              1         1       1      1
14              1         1       1      1
15              1         1       1      1
16              1         1       1      1
17              1         1       1      1
18              1         1       1      1
26              1         1       1      1
35              1         1       1      1
36              1         1       1      1
37              1         1       1      1
44              1         1       1      1
48              1         1     

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    if row['userId'] == '80':
        print(row)
    cur.execute(user_table_insert, row)
    
#cur.copy_from(user_df,'users')

cur.execute("select * from users where user_id = 80")
row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
cur.execute('delete from songplays')
conn.commit()

chk = pd.DataFrame(df[['song','artist','length','ts','userId','sessionId','level','location','userAgent']])


print(chk.count())
chk.drop_duplicates(keep=False,inplace=True)
print(chk.count())

for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    #print(row['song'])
    results = cur.fetchone()
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    #print(row['userAgent'].strip('"'))
    songplay_data = [dt.datetime.fromtimestamp(row['ts']/1000),row['userId'],row['level'],row['song'],row['artist'],row['sessionId'],row['location'],row['userAgent'].strip('"')]
    #print(songplay_data)
    # insert songplay record
    cur.execute(songplay_table_insert, songplay_data)

conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.